In [1]:
#from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
#from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

#from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
#from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

#from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender



In [2]:
import traceback, os
import scipy.sparse
URM_all = scipy.sparse.load_npz('URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('URM_test_matrix.npz')

In [4]:

recommender_list = [
    #Random,
    #TopPop,
    #P3alphaRecommender,
    #RP3betaRecommender,
    ItemKNNCFRecommender,
    #UserKNNCFRecommender,
    #MatrixFactorization_BPR_Cython,
   #MatrixFactorization_FunkSVD_Cython,
   #PureSVDRecommender,
    #SLIM_BPR_Cython,
    #SLIMElasticNetRecommender
    ]


from Base.Evaluation.Evaluator import SequentialEvaluator

#evaluator = SequentialEvaluator(URM_test, [5, 20], exclude_seen=True)


output_root_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_root_path):
    os.makedirs(output_root_path)


logFile = open(output_root_path + "result_all_algorithms.txt", "a")


for recommender_class in recommender_list:

    try:

        print("Algorithm: {}".format(recommender_class))


        recommender = recommender_class(URM_train)
        recommender.fit()

        
        results_run = recommender.evaluateRecommendations(URM_test, at=10, exclude_seen=True)
        print("Algorithm: {}, results: {}".format(recommender.__class__, results_run))

        '''
        results_run, results_run_string = evaluator.evaluateRecommender(recommender)
        print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_run_string))
        logFile.write("Algorithm: {}, results: \n{}\n".format(recommender.__class__, results_run_string))
        logFile.flush()
        '''

        
    except Exception as e:
        traceback.print_exc()
        logFile.write("Algorithm: {} - Exception: {}\n".format(recommender_class, str(e)))
        logFile.flush()



Algorithm: <class 'KNN.ItemKNNCFRecommender.ItemKNNCFRecommender'>
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 20600 ( 100 % ), 693.04 column/sec, elapsed time 0.50 min
Ignoring 0 Items
Ignoring 0 Users
SequentialEvaluator: Processed 9676 ( 100.00% ) in 8.27 seconds. Users per second: 1171
Algorithm: <class 'KNN.ItemKNNCFRecommender.ItemKNNCFRecommender'>, results: {'roc_auc': 0.2942410365689611, 'precision': 0.175797874958168, 'recall': 0.17407603696451884, 'recall_test_len': 0.175797874958168, 'map': 0.08335977693520995, 'mrr': 0.22794789267505158, 'ndcg': 0.14292065444725413, 'f1': 0.17493271911204525, 'hit_rate': 0.7549607275733774, 'arhr': 0.28643407514583713, 'novelty': 0.005686893722092346, 'diversity_mean_inter_list': 0.9874447980191272, 'diversity_herfindahl': 0.9987342747089195, 'coverage_item': 0.28776350860189, 'coverage_user': 0.19180906315664276, 'diversity_gini': 0.2373246930119993, 'shannon_entropy': 10.693815350198495}
